<a href="https://colab.research.google.com/github/aneeshcheriank/tensorflow-great_barrier_reef/blob/main/barrier_reef.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
path = files.upload()


# create a kaggle folder|
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!kaggle competitions download -c tensorflow-great-barrier-reef 

     |████████████████████████████████| 58 kB 3.2 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=05060857aeae01aec470776daadbb6eb77750efbe50642404524fe508f67d163
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
100% 14.2G/14.2G [05:27<00:00, 37.1MB/s]
100% 14.2G/14.2G [05:27<00:00, 46.6MB/s]


In [4]:
import zipfile
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil
import cv2
import matplotlib.pyplot as plt

In [5]:
# Extract the zip file
def extract(zip_file, output_path):
  os.makedirs(output_path, exist_ok=True)
  with zipfile.ZipFile(zip_file) as z:
    z.extractall(output_path)

zip_file = 'tensorflow-great-barrier-reef.zip'    
output_path = './data/extract'

extract(zip_file, output_path)

# Data Processing
- format the data columns
  - annotations were string, changed it into list
- split the data
  - data is split into train and test

In [ ]:
BASE_DIR = './data/extract'

def data_processing(df, data_type = 'train'):

  # create the folders
  image_folder = f'./data/yolo/image/{data_type}'
  label_folder = f'./data/yolo/label/{data_type}'

  for folder in (image_folder, label_folder):
    os.makedirs(folder, exist_ok=True)

  # copy image and label file into folders
  for _, row in tqdm(df.iterrows()):

    annotations = row['annotations']

    # taking only those rows with annotations
    if len(annotations) > 0:
      
      folder_id, file_id = row['image_id'].split('-')
      folder = f'video_{folder_id}'
      file = f'{file_id}.jpg'
    
      # images
      shutil.copy(
          os.path.join(BASE_DIR, f'train_images/{folder}/{file}'),
          os.path.join(image_folder, file)
      )

      # labels
      yolo_labels = []
      for annotation in annotations:
        x = annotation['x']
        y = annotation['y']
        w = annotation['width']
        h = annotation['height']

        x_center = x + w/2
        y_center = y+ h/2

        x_center /= 1280
        y_center /= 720
        w /= 1280
        h /= 720

      yolo_labels.append([0, x_center, y_center, w, h])
      np.savetxt(
          os.path.join(label_folder, f'{file_id}.txt'),
          yolo_labels,
          fmt = ['%d', '%f', '%f', '%f', '%f']
      )





if __name__ == "__main__":
  data = pd.read_csv(
      os.path.join(BASE_DIR, 'train.csv')
  ) 

  data['annotations'] = data['annotations'].apply(eval)

  train, test = train_test_split(
      data,
      test_size = 0.25,
      random_state=42,
      shuffle=True
  )

  data_processing(train, 'train')
  data_processing(test, 'test')

6886it [00:12, 693.12it/s]

In [22]:
# upload the processed data to google drive
%%bash
rm -rf tensorflow-great-barrier-reef.zip
zip -qr great-barrier-reef.zip ./data/yolo


from google.colab import drive
drive.mount('/content/drive')

!mv great-barrier-reef.zip ./drive/MyDrive/Kaggle/Data


fatal: not a git repository (or any of the parent directories): .git


In [27]:
%%bash

ls

data
drive
kaggle.json
sample_data
